In [1]:
import numpy as np
import pandas as pd
from preprocess import preprocess  # local file. restart kernel if this changed, it won't be re-imported otherwise
from sklearn.model_selection import train_test_split

## Read the first few rows during crude developing:
#train = pd.read_csv('../data/train.csv', nrows=10000).fillna(' ')  # train has 159571 comments
#test = pd.read_csv('../data/test.csv', nrows=1000).fillna(' ')

## These lines load all data:
train = pd.read_csv('../data/train.csv').fillna(' ')
test = pd.read_csv('../data/test.csv').fillna(' ')

[train, test, train_text, test_text, all_text, class_names] = preprocess(train, test)

# Remove 80% of the train set's comments that are all-ok to counter the imbalancedness a bit
#delete_idx = (train[class_names].sum(axis = 1) == 0) & (np.random.rand(len(train)) > 0.2)
#keep_idx = ~delete_idx  # elem-wise NOT for pd.Series
#train = train[keep_idx]

In [2]:
# I want a fixed 5000 observations in the test set. Adapt the fraction accordingly:
test_fraction = 5000 / len(train)

train, dev = train_test_split(train, test_size=test_fraction, random_state=42)
# Reset indices because they were shuffled and I need them from 0 to len()
train.index = list(range(len(train)))
dev.index = list(range(len(dev)))

In [3]:
# either:
#max_len_train = train['comment_text'].str.split().apply(len).max()
#max_len_test = test['comment_text'].str.split().apply(len).max()
#max_len = max(max_len_train, max_len_test)

# or:
max_len = 200  # that might work to have shorter LSTM cells

In [4]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, GRU, Activation, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../data/glove.6B.100d.txt')

In [7]:
import re

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]

    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
        
        # Convert the ith training sentence in lower case and split is into words. You get a list of words.
        sentence_words = re.sub(r"\W", " ", X[i]).lower().split()

        n_words = len(sentence_words)
        
        if n_words > max_len:  # use the *beginning* of the comment if it's too long
            sentence_words = sentence_words[0:max_len]
            n_words = max_len

        ctr = 0
        # Loop over the words of sentence_words
        for w in sentence_words:
            j = max_len - n_words + ctr 
            
            if w in word_to_index:
                # pad sequences from the left, i.e. start so the "last" word in the comment is the max_len'th entry.
                X_indices[i, j] = word_to_index[w]
                
            ctr += 1
            
            if j >= max_len:  
                # adding this clause, you can have sentences longer than max_len and crop them
                break
            
    return X_indices

In [8]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (e.g. 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)#, mask_zero = True)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [9]:
def myModel(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    X = embedding_layer(sentence_indices)   

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    #X = LSTM(units = 64, return_sequences = True)(X)
    X = Bidirectional(GRU(units = 32, activation = 'relu', return_sequences = True, 
                          dropout = 0.25, recurrent_dropout = 0.25))(X)
    X_max = GlobalMaxPooling1D()(X)
    X_avg = GlobalAveragePooling1D()(X)
    X = concatenate([X_max, X_avg])
    X = Dense(50, activation = 'relu')(X)
    # Add dropout with a probability of 0.5
    X = Dropout(rate = 0.25)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    #X = LSTM(units = 64)(X)
    #X = GRU(units = 64, activation = 'relu')(X)
    # Add dropout with a probability of 0.5
    #X = Dropout(rate = 0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(units = 6, activation = 'sigmoid')(X)
    # Add a softmax activation
    #X = Activation("softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [10]:
model = myModel((max_len,), word_to_vec_map, word_to_index)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 100)     40000100    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 200, 64)      25536       embedding_1[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 64)           0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_ave

In [11]:
X_train_indices = sentences_to_indices(train['comment_text'], word_to_index, max_len)
Y_train = train[class_names].values
X_dev_indices = sentences_to_indices(dev['comment_text'], word_to_index, max_len)
Y_dev = dev[class_names].values

In [17]:
# If you want gradient clipping, put optimizer=adam_gradclip in the model.compile() call

#from keras import optimizers
#
#adam_gradclip = optimizers.Adam(clipnorm = 1, clipvalue = 0.5)

In [22]:
# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# if you want to save a model's configuration and weights, and train it further tomorrow:

from keras.models import load_model
model = load_model("10-model.h5")

## Loss history:

Epoch 1/5
154571/154571 [==============================] - 159s 1ms/step - loss: 0.1179 - acc: 0.9644

Epoch 2/5
154571/154571 [==============================] - 995s 6ms/step - loss: 0.0642 - acc: 0.9781

Epoch 3/5
154571/154571 [==============================] - 1093s 7ms/step - loss: 0.0586 - acc: 0.9793

Epoch 4/5
154571/154571 [==============================] - 1098s 7ms/step - loss: 0.0560 - acc: 0.9799

Epoch 5/5
154571/154571 [==============================] - 1153s 7ms/step - loss: 0.0540 - acc: 0.9805

CPU times: user 5h 9min 36s, sys: 2h 22min 2s, total: 7h 31min 38s
Wall time: 1h 15min


Epoch 1/5
154571/154571 [==============================] - 866s 6ms/step - loss: 0.0525 - acc: 0.9808

Epoch 2/5
154571/154571 [==============================] - 1166s 8ms/step - loss: 0.0513 - acc: 0.9813

Epoch 3/5
154571/154571 [==============================] - 1159s 7ms/step - loss: 0.0507 - acc: 0.9815

Epoch 4/5
154571/154571 [==============================] - 1097s 7ms/step - loss: 0.0498 - acc: 0.9816

Epoch 5/5
154571/154571 [==============================] - 1163s 8ms/step - loss: 0.0490 - acc: 0.9818

CPU times: user 6h 17s, sys: 3h 3min 43s, total: 9h 4min

Wall time: 1h 30min 50s

In [ ]:
%%time 

model.fit(X_train_indices, Y_train, epochs = 5, batch_size = 512, shuffle=True)
model.save("10-model.h5")

Epoch 1/5
154571/154571 [==============================] - 556s 4ms/step - loss: 0.0485 - acc: 0.9819
Epoch 2/5
154571/154571 [==============================] - 1247s 8ms/step - loss: 0.0478 - acc: 0.9821
Epoch 3/5
 47616/154571 [========>.....................] - ETA: 11:57 - loss: 0.0477 - acc: 0.9821

In [ ]:
from sklearn import metrics

m = 5000

pred_train = model.predict(X_train_indices[0:m, :])  # train performance on first 5k samples only...
aucs = np.zeros((len(class_names)))

for y_i in range(len(class_names)):
    y = Y_train[0:m, y_i]
    yhat = pred_train[0:m, y_i]
    fpr, tpr, thresholds = metrics.roc_curve(y, yhat, pos_label = 1)
    this_auc = metrics.auc(fpr, tpr)
    aucs[y_i] = this_auc
    
np.mean(aucs)

Glove 100 w/ dropout (batch size 512), avg+max pooling concat'ed:

- 5 epochs: 0.9813304560865536
- 10: 0.9858895978606

Glove 100 w/ dropout (batch size 512):

- 5 epochs: 0.9806128728968982
- 10: 0.9858491355190292

Glove 100 w/o dropout:

- 1 epoch: 0.9855825964913917
- 2: 0.9883302986173689

# Compute dev set mean AUC

In [ ]:
# %%time

# model.evaluate(X_dev_indices, Y_dev, batch_size = len(train))
pred_dev = model.predict(X_dev_indices)

aucs = np.zeros((len(class_names)))

for y_i in range(len(class_names)):
    y = Y_dev[:, y_i]
    yhat = pred_dev[:, y_i]
    fpr, tpr, thresholds = metrics.roc_curve(y, yhat, pos_label = 1)
    this_auc = metrics.auc(fpr, tpr)
    aucs[y_i] = this_auc
    
np.mean(aucs)

Glove 100 w/ dropout (batch size 512), avg+max pooling concat'ed:

- 5: 0.9818096375442297
- 10: 0.9868617031269369  # 0.9765 on kaggle


Glove 100 w/ dropout (batch size 512):

- 5 epochs: 0.9794802200593979
- 10: 0.985146911642742  # 0.9738 on kaggle

Glove 100 w/o dropout:

- 1 epoch: 0.9832965881854069
- 2: 0.986526510845655  # 0.9724 on kaggle


# Sanity check train predictions

In [ ]:
pred = model.predict(X_train_indices[0:7, :])  # the 7th comment (in unshuffled train) is a very toxic doggo

In [ ]:
pred

# Create submission csv

In [ ]:
X_test_indices = sentences_to_indices(test['comment_text'], word_to_index, max_len)
pred = model.predict(X_test_indices)
submit = pd.DataFrame(test['id'].values, columns=['id'])
preds = pd.DataFrame(pred, columns=class_names)
submit = pd.concat([submit, preds], axis = 1)
#submit
submit.to_csv('../data/submission_10.csv', index = False)